In [69]:
import builtins
from typing import *

class GlobalFileInput:
    
    def __init__(self, file_path='input.txt'):

        self.file_path = file_path
        self.file = None
        self.original_input = builtins.input
    
    def start(self):
        self.file = open(self.file_path, 'r')
        builtins.input = self.file_input

    def stop(self):
        if self.file:
            builtins.input = self.original_input
            self.file.close()

    def file_input(self, prompt=''):
        return self.file.readline().strip()

s = GlobalFileInput('input.txt')

In [70]:
import google.generativeai as genai

genai.configure(api_key="YOUR KEY")
model = genai.GenerativeModel("gemini-1.5-flash")

In [71]:
import pandas as pd


tenants = pd.DataFrame({
    "tenant_id": [1, 2],
    "name": ["Alice", "Bob"],
    "contact_info": ["alice@example.com", "bob@example.com"]
})

payments = pd.DataFrame({
    "payment_id": [1, 2],
    "tenant_id": [1, 2],
    "amount": [1500, 1400],
    "status": ["Paid", "Pending"],
    "last_updated": ["2024-12-01", "2024-12-15"]
})


print("Tenants Table:")
print(tenants)

print("\nPayments Table:")
print(payments)


Tenants Table:
   tenant_id   name       contact_info
0          1  Alice  alice@example.com
1          2    Bob    bob@example.com

Payments Table:
   payment_id  tenant_id  amount   status last_updated
0           1          1    1500     Paid   2024-12-01
1           2          2    1400  Pending   2024-12-15


In [82]:
def generate_sql_from_prompt(user_query, tenants, payments):

    tenants_table = tenants
    payments_table = payments

    prompt = f"""
    Below are two tables: tenants and payments.

    tenants Table:
    {tenants_table.to_string(index=False)}

    payments Table:
    {payments_table.to_string(index=False)}

    Translate the following user question into a valid SQL query and return the result of that query that is fetch that value from the table and.:

    Question: "{user_query}"
    SQL Query:
    """
    
    response = model.generate_content(prompt)
    sql_query = response.text.strip()
    
    sql_query = sql_query.replace("```sql", "").replace("```", "").strip()
    
    return sql_query


In [73]:
import pandasql as ps

def execute_sql_with_pandas(sql_query,tenants,payments):
   
    result = ps.sqldf(sql_query, locals())
    return result


In [80]:
s.start()

user_query = input()

sql_query = generate_sql_from_prompt(user_query, tenants, payments)

print(sql_query)
    
result = execute_sql_with_pandas(sql_query, tenants, payments)

print()
print(result)


SELECT
    p.last_updated,
    t.contact_info
FROM
    payments p
JOIN
    tenants t ON p.tenant_id = t.tenant_id
WHERE
    t.name = 'Bob' OR t.name = 'Alice';

  last_updated       contact_info
0   2024-12-01  alice@example.com
1   2024-12-15    bob@example.com


In [84]:
s.start()

user_query = input()

sql_query = generate_sql_from_prompt(user_query, tenants, payments)

print(sql_query)


SELECT
  p.last_updated,
  t.contact_info,
  p.status
FROM tenants t
JOIN payments p ON t.tenant_id = p.tenant_id
WHERE
  t.name = 'Bob' OR t.name = 'Alice';


This query will return a result set with three columns: `last_updated`, `contact_info`, and `status`.  It will include a row for Bob showing his last updated payment date, and a separate row for Alice showing her contact information and payment status.


**Result:**

| last_updated | contact_info      | status   |
|--------------|----------------------|----------|
| 2024-12-15   | bob@example.com     | Pending  |
| 2024-12-01   | alice@example.com   | Paid     |
